<a href="https://colab.research.google.com/github/DmitryKutsev/congrats_generator/blob/feature%233/my_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [1]:
import pandas as pd

In [2]:
!pip install transformers -U

In [3]:
!pip install sentencepiece

In [4]:
!pip install tqdm boto3 requests regex sacremoses

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [6]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [7]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [8]:
!pip install tensorboard
# !pip install wandb; wandb login

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
!pip install awscli
!aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2

download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/pytorch_model.bin to gpt2/s_checkpoint-1900000/pytorch_model.bin
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/pytorch_model.bin to gpt2/m_checkpoint-3364613/pytorch_model.bin


# **PREPARE DATA**

In [11]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt

--2021-12-20 19:27:31--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt’

curr_congrats.txt   100%[===================>]   1.40M  --.-KB/s    in 0.05s   

2021-12-20 19:27:31 (30.1 MB/s) - ‘curr_congrats.txt’ saved [1471875/1471875]



In [12]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [13]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [14]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')
  # print(letter)
  # print(letter.split('CONTENT'))
  # print(letter.split('CONTENT'))
  # break
  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [15]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [16]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

In [18]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [19]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [20]:
! pip install datasets transformers

     |████████████████████████████████| 298 kB 6.4 MB/s 
     |████████████████████████████████| 132 kB 58.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 243 kB 38.2 MB/s 
     |████████████████████████████████| 192 kB 58.4 MB/s 
     |████████████████████████████████| 160 kB 54.1 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 


In [31]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [26]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}


In [33]:
train_test_ratio = 0.9
df_full_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [34]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Title'].tolist() + df['Content'].tolist()
    # c = 0
    # if c < 2:
    #   print(summaries)
    #   c += 1
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        bos_token = '<BOS>'
        eos_token = '<EOS>'
        data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        
    f.write(data)

In [36]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

In [32]:
# !export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
# !CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
#     --output_dir=models/essays \
#     --model_type=gpt2 \
#     --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
#     --do_train \
#     --train_data_file=train.txt \
#     --do_eval \
#     --eval_data_file=valid.txt \
#     --per_gpu_train_batch_size 1 \
#     --gradient_accumulation_steps 1 \
#     --num_train_epochs 5 \
#     --block_size 2048 \
#     --overwrite_output_dir

12/20/2021 20:24:08 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
Downloading: 100% 608/608 [00:00<00:00, 586kB/s]
Downloading: 100% 1.63M/1.63M [00:00<00:00, 3.95MB/s]
Downloading: 100% 1.21M/1.21M [00:00<00:00, 3.31MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Traceback (most recent call last):
  File "ru-gpts/pretrain_transformers.py", line 782, in <module>
    main()
  File "ru-gpts/pretrain_transformers.py", line 698, in main
    args.block_size = min(args.block_size, tokenizer.max_len)
AttributeError: 'GPT2TokenizerFast' object has no attribute 'max_len'


In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")

train_path = 'train.txt'
test_path = 'test.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# gpt = RuGPT3XL.from_pretrained(
#     "sberbank-ai/rugpt3xl",
#     weights_path="/path/2/save/model/train-iters/mp_rank_00_model_states.pt",
#     deepspeed_config_path="../src/deepspeed_config/gpt3_xl_sparse_2048.json",
#     seq_len=512
# )

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt2large")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fa80c74fb50>> (for pre_run_cell):


Exception: ignored

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:746: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
https://huggingface.co/sberbank-ai/rugpt2large/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpea649vt0


Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

storing https://huggingface.co/sberbank-ai/rugpt2large/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c9a8c0802df5cb1baa68939b4caf94e65ce7645ec7452a9e4f1071f84d696d82.a964e5b8fd85ed3f0915661e4ff4502227e219a4e108d2add83b42afdebcd877
creating metadata file for /root/.cache/huggingface/transformers/c9a8c0802df5cb1baa68939b4caf94e65ce7645ec7452a9e4f1071f84d696d82.a964e5b8fd85ed3f0915661e4ff4502227e219a4e108d2add83b42afdebcd877
loading configuration file https://huggingface.co/sberbank-ai/rugpt2large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c9a8c0802df5cb1baa68939b4caf94e65ce7645ec7452a9e4f1071f84d696d82.a964e5b8fd85ed3f0915661e4ff4502227e219a4e108d2add83b42afdebcd877
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt2large",
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 

Downloading:   0%|          | 0.00/3.16G [00:00<?, ?B/s]

In [42]:
trainer.train()

***** Running training *****
  Num examples = 4451
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 420
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model()